# Práctica 3 - Reducción de dimensionalidad

__Curso__: Statistical Learning II

__Catedrático__: Ing. Luis Leal

__Estudiante__: Dany Rafael Díaz Lux (21000864)

In [94]:
# Importar librerías que utilizaremos
#from scipy.stats import multivariate_normal
#from sklearn.metrics import f1_score
from scipy import stats
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
#import matplotlib.pyplot as plt
import numpy as np
#import pandas as pd
#import random

## Cargar y escalar información

In [3]:
# Cargar imágenes de fashion mnist
fashion_mnist = keras.datasets.fashion_mnist
(imagenes_train, etiquetas_train), (imagenes_test, etiquetas_test) = fashion_mnist.load_data()

In [16]:
# Pasar de matrices de 28x28 a vectores de 784.
imagenesPlanas = imagenes_train.reshape(imagenes_train.shape[0], imagenes_train.shape[1] * imagenes_train.shape[2])
imagenesPlanas = np.append(imagenesPlanas, \
                           imagenes_test.reshape(imagenes_test.shape[0], imagenes_test.shape[1] * imagenes_test.shape[2]), axis= 0)
etiquetas = np.append(etiquetas_train, etiquetas_test)
# Estandarizar información
estandarizadorImagenes = StandardScaler()
imagenesPlanas = estandarizadorImagenes.fit_transform(imagenesPlanas)
print('Dimensiones de imágenes')
print(imagenesPlanas.shape)
print(imagenesPlanas[0:5,:])
print('Número de etiquetas')
print(etiquetas.shape)
print(etiquetas[0:5])

Dimensiones de imágenes
(70000, 784)
[[-0.00883265 -0.02162585 -0.0287172  ... -0.15825699 -0.09035386
  -0.03423352]
 [-0.00883265 -0.02162585 -0.0287172  ... -0.15825699 -0.09035386
  -0.03423352]
 [-0.00883265 -0.02162585 -0.0287172  ... -0.15825699 -0.09035386
  -0.03423352]
 [-0.00883265 -0.02162585 -0.0287172  ... -0.15825699 -0.09035386
  -0.03423352]
 [-0.00883265 -0.02162585 -0.0287172  ... -0.15825699 -0.09035386
  -0.03423352]]
Número de etiquetas
(70000,)
[9 0 0 3 0]


## Análisis de componentes principales (PCA)

In [73]:
#test = np.array([[4, 2, 1, 3], [7, 4, 2, 6], [10, 6, 3, 9], [24, 20, 10, 30], [18, 14, 7, 21]])
#cov = np.cov(test.T)
#print(cov)
#vals, vecs = np.linalg.eig(cov)
#print(vals)
#indices = np.argsort(-np.abs(vals))[0:2]
#print(indices)
#print(vecs)
#print(vecs[:,indices])
#from sklearn.decomposition import PCA
#np.random.seed(0)
#my_matrix = np.random.randn(20,5)
#cov = np.cov(my_matrix.T)
#print(cov.diagonal())
#print(cov.diagonal()/np.sum(cov.diagonal()))
#my_model = PCA(n_components=5)
#my_model.fit_transform(my_matrix)
#print(my_model.explained_variance_ratio_)
#print(np.sum(np.abs(vals)))
#test2 = sorted(vals, reverse=True)/np.sum(vals)
#print(test2.T)
#print(vals)
#total_egnvalues = sum(vals)
#var_exp = np.array([(i/total_egnvalues) for i in sorted(vals, reverse=True)])
#print(var_exp)
#print(var_exp.cumsum()[0])

In [77]:
# Función que devolverá la matriz de "k" dimensiones reducida para X
def obtenerMatrizReducidaPCA(X, k):
    if(k < 1 or k > X.shape[1]):
        print('Error enviado en el número de dimensiones que se desea preservar k: ', k)
        return None
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
    # Obtener matriz de covarianza
    matrizCovarianza = np.cov(X.T)
    # Obtener eigen-values y eigen-vectors
    eigenVals, eigenVecs = np.linalg.eig(matrizCovarianza)
    # Determinar cantidad de información preservada
    varianzaPreservada = sorted(eigenVals, reverse=True)/np.sum(eigenVals)
    informacionPreservada = varianzaPreservada.cumsum()[k-1]
    # Determinar los índices de los "k" eigen-values con valores absolutos más grandes.
    indices = np.argsort(-np.abs(eigenVals))[0:k]
    # Devolver la matriz con los k eigen vectors
    return eigenVecs[:,indices], informacionPreservada

# Función para transformar información en dimensiones reducidas
def transformarConMatrizReducidaPCA(X, matrizReducida):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)
    if(type(matrizReducida).__module__ != np.__name__):
        matrizReducida = np.array(matrizReducida)
    # Asegurarse que el número de características sean las columnas de X y el número de filas de matrizReducida
    if(X.shape[1] == matrizReducida.shape[0]):
        return np.matmul(X, matrizReducida)
    print('Error en las dimensiones de las matrices')
    print('Forma de datos')
    print(X.shape)
    print('Forma matriz reducida')
    print(matrizReducida.shape)
    return None

## t-Distributed Stochastic Neighbor Embedding (t-SNE)

In [78]:
def obtenerModeloTsne(X, k):
    return TSNE(n_components=k, random_state=2022).fit_transform(X)

## Reducir información a 2 dimensiones con PCA y t-SNE

In [82]:
# Se realizarán sólo las primeras 10,000 por cuestiones de tiempo
# Reducir con PCA
matrizReducida, informacionPreservada = obtenerMatrizReducidaPCA(imagenesPlanas[0:10000,:], 2)
imagenesReducidasConPCA = transformarConMatrizReducidaPCA(imagenesPlanas[0:10000,:], matrizReducida)
print('Representación imágenes reducidas con PCA')
print(imagenesReducidasConPCA.shape)
print(imagenesReducidasConPCA)
print('Información preservada con PCA')
print(informacionPreservada)

Representación imágenes reducidas con PCA
(10000, 2)
[[ -0.98941186 -20.75232521]
 [ 17.00505454   4.87364139]
 [ -9.49313162  12.41609182]
 ...
 [ 13.45476333   7.77806339]
 [ -2.94757494  11.08942227]
 [ 11.72563831  -5.26358477]]
Información preservada con PCA
0.35810306937433095


In [83]:
# Se realizarán sólo las primeras 10,000 por cuestiones de tiempo
imagenesReducidasTsne = obtenerModeloTsne(imagenesPlanas[0:10000,:], 2)
print('Representación imágenes reducidas con t-SNE')
print(imagenesReducidasTsne.shape)
print(imagenesReducidasTsne)

Representación imágenes reducidas con t-SNE
(10000, 2)
[[ 59.906467  -20.277704 ]
 [-39.80932    37.376087 ]
 [-20.463747   -2.8083603]
 ...
 [-47.898346    7.2352967]
 [-21.387966   20.682526 ]
 [ 18.58468    34.523937 ]]


## Comentarios en reducción de dimensionalidad con PCA y t-SNE

* Las representaciones no muestran ningún tipo de similitud numérica (al menos no las desplegadas en notebook).
* Para PCA, la varianza conservada es de 35.81% que es mucha más de la que se esperaba dado que se redujo de 784 a 2 dimensiones.
* Para este caso específico de reducción de 784 a 2 dimensiones, el método de PCA implementado manualmente, fue más rápido que el método de t-SNE implementado por scikit-learn.

## Realizar clustering con GMM sobre representaciones reducidas (PCA)

In [114]:
print(etiquetas[0:10])
print(len(etiquetas[etiquetas == 0]))


[9 0 0 3 0 2 7 2 5 5]
7000


In [115]:
# función para crear un modelo Gaussian Mixture Model
def obtenerGMM(X, k):
    if(type(X).__module__ != np.__name__):
        X = np.array(X)    
    if(k < X.shape[0]):
        return GaussianMixture(n_components=k, random_state=2022).fit(X)
    else:
        print('k debe ser menor al número de observaciones en X.')
        return None
    
# Función que evaluará la "exactitud" de los clusters con las etiquetas
# Aspectos que tomará en cuenta
# 1. % de cluster que más aparece en una etiqueta.
# 2. Si el cluster ya ha sido escogido en otra etiqueta anterior, ese cluster tendrá una exactitud del 0%
# 3. Se devolverá porcentaje por cada etiqueta.
def calcularExactitudClusters(predicciones, etiquetas):
    etiquetasUnicas = np.unique(etiquetas)
    exactitudes = np.zeros(etiquetasUnicas.shape[0])
    prediccionesUtilizadas = []
    for iEtiqueta, etiquetaUnica in enumerate(etiquetasUnicas):
        indicesEtiqueta = np.argwhere(etiquetas == etiquetaUnica)
        prediccionesEnEtiqueta = predicciones[indicesEtiqueta]
        moda = stats.mode(prediccionesEnEtiqueta)[0][0]
        # Agregar moda obtenida en predicciones utilizadas
        if(moda not in prediccionesUtilizadas):
            prediccionesUtilizadas.append(moda)
            exactitudes[iEtiqueta] = (len(prediccionesEnEtiqueta[prediccionesEnEtiqueta == moda]) / len(prediccionesEnEtiqueta))
        else:
            exactitudes[iEtiqueta] = 0.0
    return exactitudes

In [116]:
# Para PCA
modeloGMM = obtenerGMM(imagenesReducidasConPCA, 10)
exactitudes = calcularExactitudClusters(modeloGMM.predict(imagenesReducidasConPCA), etiquetas[0:10000])
print('Promedio de exactitud obtenido de los clusters con datos con dimensionalidad reducida con PCA:')
print(np.mean(exactitudes))
# Para t-SNE
modeloGMM = obtenerGMM(imagenesReducidasTsne, 10)
exactitudes = calcularExactitudClusters(modeloGMM.predict(imagenesReducidasTsne), etiquetas[0:10000])
print('Promedio de exactitud obtenido de los clusters con datos con dimensionalidad reducida con t-SNE:')
print(np.mean(exactitudes))

Promedio de exactitud obtenido de los clusters con datos con dimensionalidad reducida con PCA:
0.39064800076142137
Promedio de exactitud obtenido de los clusters con datos con dimensionalidad reducida con t-SNE:
0.5178117204311274


## Conclusiones
* La exactitud (definida en los comentarios de la función) de los clusters para los datos reducidos con PCA fue de un 39.06%.
* La exactitud de los clusters para los datos reducidos con t-SNE fue de un 51.78%
* Dado que se pasó de 784 a 2 dimensiones, las métricas de exactitud parecen bastante altas al tomar en cuenta la gran diferencia en reducción de dimensionalidad que se llevó a cabo.
* Se puede observar cierto acercamiento con la cantidad de varianza preservada con la definición de exactitud definida en la función para el método de PCA. Más experimentación sería necesaria para determinar si existe una correlación fuerte entre ambas.